# Part 2: Power Method

In [2]:
import numpy as np

#### 1. Implement the power method with normalization.

In [34]:
def power_method(matrix: np.ndarray, maximum_iterations: int):
    x_k = np.random.rand(matrix.shape[1])
    lmbda = 0
    # deze werkt nog niet, fix komt eraan
    for _ in range(maximum_iterations):
        z_k1 = np.dot(matrix, x_k)
        x_k = z_k1 / np.linalg.norm(z_k1)
        i = np.argmax(z_k1)
        lmbda = z_k1[i]/x_k[i]

    return lmbda, x_k
A = np.array([[5, -1, -2, 0], [-1, 3, -2, -1], [-2, -2, 5, 0], [0, -1, 0, 4]])
eigenvalue, eigenvector = power_method(A, 100)
print(f"Largest eigenvalue: {eigenvalue}, associated eigenvector: {eigenvector}")
print(f"Ax = {A @ eigenvector} = lambda x = {eigenvalue * eigenvector}")

Largest eigenvalue: 7.179335040371524, associated eigenvector: [ 0.59084255  0.24322828 -0.76543608 -0.07650288]
Ax = [ 4.24185663  1.74621733 -5.49532206 -0.5492398 ] = lambda x = [ 4.24185663  1.74621733 -5.49532206 -0.5492398 ]


#### 2. Extend your function, to a new one, such that it takes an extra argument $\epsilon = 10^{−5}$. Keep track on the number of iterations that you need to reach the precision $|λ(k+1) − λ(k)| ≤ ε$.


In [51]:
def power_method_limit(matrix: np.ndarray, maximum_iterations: int, epsln: float):
    x_k = np.random.rand(matrix.shape[1])
    oldlmbda = 0
    lmbda = 0
    itr = 0
    for _ in range(maximum_iterations):
        z_k1 = np.dot(matrix, x_k)
        x_k = z_k1 / np.linalg.norm(z_k1)
        i = np.argmax(z_k1)
        oldlmbda = lmbda
        lmbda = z_k1[i]/x_k[i]
        itr += 1
        if (np.abs(lmbda - oldlmbda) <= epsln):
            break

    return lmbda, x_k, itr
A = np.array([[5, -1, -2, 0], [-1, 3, -2, -1], [-2, -2, 5, 0], [0, -1, 0, 4]])
eigenvalue_lim, eigenvector_lim, iterations_lim = power_method_limit(A, 100, 1e-5)
print(f"Largest eigenvalue: {eigenvalue_lim}, associated eigenvector: {eigenvector_lim}, iterations: {iterations_lim}")
print(f"Ax = {A @ eigenvector_lim} = lambda x = {eigenvalue_lim * eigenvector_lim}")


Largest eigenvalue: 7.179323467297241, associated eigenvector: [-0.58965722 -0.24448586  0.76585573  0.07743396], iterations: 27
Ax = [-4.23551172 -1.7529458   5.49756483  0.55422172] = lambda x = [-4.23333994 -1.75524311  5.49832603  0.55592348]


3.


In [52]:
def power_method_inverse(matrix: np.ndarray, maximum_iterations: int, epsln: float):
    inverse_matrix = np.linalg.inv(matrix)
    x_k = np.random.rand(inverse_matrix.shape[1])
    oldlmbda = 0
    lmbda = 0
    itr = 0
    
    for _ in range(maximum_iterations):
        z_k1 = np.dot(inverse_matrix, x_k)
        x_k = z_k1 / np.linalg.norm(z_k1)
        i = np.argmax(z_k1)
        oldlmbda = lmbda
        lmbda = z_k1[i]/x_k[i]
        itr += 1
        if (np.abs(lmbda - oldlmbda) <= epsln):
            break
    smallest_lbmda = 1/lmbda
    return smallest_lbmda, x_k, itr
A = np.array([[5, -1, -2, 0], [-1, 3, -2, -1], [-2, -2, 5, 0], [0, -1, 0, 4]])
eigenvalue_inv, eigenvector_inv, iterations_inv = power_method_inverse(A, 100, 1e-5)
print(f"Largest eigenvalue: {eigenvalue_inv}, associated eigenvector: {eigenvector_inv}, iterations: {iterations_inv}")
print(f"Ax = {A @ eigenvector_inv} = lambda x = {eigenvalue_inv * eigenvector_inv}")

Largest eigenvalue: 0.6800134536157676, associated eigenvector: [0.40747751 0.71807889 0.52110163 0.21628194], iterations: 5
Ax = [0.27710541 0.48827395 0.35439536 0.14704886] = lambda x = [0.27709019 0.4883033  0.35435612 0.14707463]


4.

In [64]:
def power_method_shifted(matrix: np.ndarray, maximum_iterations: int, epsln: float, shift: int):
    shifted_matrix = matrix - shift
    x_k = np.random.rand(shifted_matrix.shape[1])
    oldlmbda = 0
    lmbda = 0
    itr = 0
    
    for _ in range(maximum_iterations):
        z_k1 = np.dot(shifted_matrix, x_k)
        x_k = z_k1 / np.linalg.norm(z_k1)
        i = np.argmax(z_k1)
        oldlmbda = lmbda
        lmbda = z_k1[i]/x_k[i]
        itr += 1
        if (np.abs(lmbda - oldlmbda) <= epsln):
            break
    furthest_lmbda = lmbda + shift
    return furthest_lmbda, x_k, itr
A = np.array([[5, -1, -2, 0], [-1, 3, -2, -1], [-2, -2, 5, 0], [0, -1, 0, 4]])
eigenvalue_shift, eigenvector_shift, iterations_shift = power_method_shifted(A, 100, 1e-5, 3)
print(f"Largest eigenvalue: {eigenvalue_shift}, associated eigenvector: {eigenvector_shift}, iterations: {iterations_shift}")
print(f"Ax = {A @ eigenvector_shift} = lambda x = {eigenvalue_shift * eigenvector_shift}")

Largest eigenvalue: 13.891316940725346, associated eigenvector: [-0.47600324 -0.56914012 -0.50655879 -0.43920227], iterations: 11
Ax = [-0.79775851  0.22090274 -0.44250722 -1.18766897] = lambda x = [-6.61231187 -7.90610576 -7.03676867 -6.10109797]
